# User Item Biases With Regularization
* Prediction for user $i$ and item $j$ is $\tilde r_{ij} = u_i + a_j$
* Loss function is $L = \sum_{\Omega}w_{ij}(r_{ij} - u_i - a_j)^2 + \lambda_u \sum_i (u_i - \bar u) ^2 + \lambda_a \sum_j (a_j - \bar a)^2 $
* $\bar u$ is the mean of $u_i$ and $\bar a$ is the mean of $a_j$ 
* $\Omega$ is the set of oberved pairs $(i, j)$
* $r_{ij}$ is the rating for user $i$ and item $j$
* $w_{ij}$ is the weight for the prediction $r_{ij}$

In [1]:
const name = "UserItemBiases"
const residual_alphas = []
const validation_weight_scheme = "inverse"

"inverse"

In [2]:
using NBInclude
@nbinclude("Alpha.ipynb");

In [3]:
const training = get_residuals("training", residual_alphas)
const validation = get_residuals("validation", residual_alphas);

Progress: 100%|███████████████████████████| Time: 0:00:00 ( 1.43 μs/it)


## Alternating Least Squares Algorithm
* $u_i = \dfrac{\sum_{j \in \Omega_i}(r_{ij} - a_j) w_{ij} + \bar u \lambda_u}{ \sum_{j \in \Omega_i} w_{ij} + \lambda_u}$
* $\Omega$ is the set of (user, item) pairs that we have ratings for
* $\Omega_i$ is subset of $\Omega$ for which the user is the $i$-th user

In [4]:
function get_residuals!(users, items, ratings, weights, a, ρ, Ω)
    for row = 1:length(users)
        i = users[row]
        j = items[row]
        r = ratings[row]
        w = weights[row]
        ρ[i] += (r - a[j]) * w
        Ω[i] += w
    end
    ρ, Ω
end

function thread_range(n)
    tid = Threads.threadid()
    nt = Threads.nthreads()
    d, r = divrem(n, nt)
    from = (tid - 1) * d + min(r, tid - 1) + 1
    to = from + d - 1 + (tid ≤ r ? 1 : 0)
    from:to
end

function update_users!(users, items, ratings, weights, u, a, λ_u, ρ, Ω)
    Threads.@threads for t = 1:Threads.nthreads()
        range = thread_range(length(ratings))
        ρ[:, Threads.threadid()] .= 0
        Ω[:, Threads.threadid()] .= 0
        @views get_residuals!(
            users[range],
            items[range],
            ratings[range],
            weights[range],
            a,
            ρ[:, Threads.threadid()],
            Ω[:, Threads.threadid()],
        )
    end

    ρ = sum(ρ, dims = 2)
    Ω = sum(Ω, dims = 2)

    μ = mean(u)
    Threads.@threads for i = 1:length(u)
        u[i] = (ρ[i] + μ * λ_u) / (Ω[i] + λ_u)
    end
end;

In [5]:
function train_model(training, λ_u, λ_a, λ_w, stop_criteria)
    @info "training model with parameters [$λ_u, $λ_a, $λ_w]"
    users, items, ratings = training.user, training.item, training.rating
    weights = get_weights("training", log(λ_w))
    u = zeros(eltype(λ_u), maximum(users))
    a = zeros(eltype(λ_a), maximum(items))

    ρ_u = zeros(eltype(u), length(u), Threads.nthreads())
    Ω_u = zeros(eltype(u), length(u), Threads.nthreads())
    ρ_a = zeros(eltype(a), length(a), Threads.nthreads())
    Ω_a = zeros(eltype(a), length(a), Threads.nthreads())

    while !stop!(stop_criteria, [u, a])
        update_users!(users, items, ratings, weights, u, a, λ_u, ρ_u, Ω_u)
        update_users!(items, users, ratings, weights, a, u, λ_a, ρ_a, Ω_a)
    end
    u, a
end;

In [6]:
function make_prediction(users, items, u, a)
    r = zeros(eltype(u), length(users))
    u_mean = mean(u)
    a_mean = mean(a)
    for i = 1:length(r)
        if users[i] > length(u)
            r[i] += mean(u)
        else
            r[i] += u[users[i]]
        end
        if items[i] > length(a)
            r[i] += mean(a)
        else
            r[i] += a[items[i]]
        end
    end
    r
end;

## Training

In [7]:
function validation_mse(λ)
    λ = exp.(λ) # ensure λ is nonnegative
    stop_criteria = convergence_stopper(1e-6, max_iters = 16)
    u, a = train_model(training, λ..., stop_criteria)
    pred_score = make_prediction(validation.user, validation.item, u, a)
    weights = get_weights("validation", validation_weight_scheme)
    mse(validation.rating, pred_score, weights)
end;

In [8]:
# Find the best regularization hyperparameters
res = optimize(
    validation_mse,
    fill(0.0f0, 3),
    LBFGS(),
    autodiff = :forward,
    Optim.Options(show_trace = true, extended_trace = true),
);
λ = exp.(Optim.minimizer(res));

[ Info: 20220508 17:06:55 training model with parameters [Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0,1.0,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0,0.0,1.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0,0.0,0.0,1.0)]
Progress: 100%|███████████████████████████| Time: 0:00:04 ( 0.47 μs/it)


Iter     Function value   Gradient norm 
     0     1.820281e+00     1.945127e-02
 * Current step size: 1.0
 * time: 0.021981000900268555
 * g(x): Float32[-0.0050086486, -1.0955863f-6, 0.019451266]
 * x: Float32[0.0, 0.0, 0.0]


[ Info: 20220508 17:07:22 training model with parameters [Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0050212,1.0050212,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0000011,0.0,1.0000011,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.9807367,0.0,0.0,0.9807367)]
Progress: 100%|███████████████████████████| Time: 0:00:04 ( 0.44 μs/it)
[ Info: 20220508 17:07:43 training model with parameters [Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0253595,1.0253595,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0000055,0.0,1.0000055,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.9073234,0.0,0.0,0.9073234)]
Progress: 100%|███████████████████████████| Time: 0:00:04 ( 0.44 μs/it)
Progress: 100%|███████████████████████████| Time: 0:00:04 ( 0.44 μs/it)
[ Info: 20220508 17:09:51 training model with parameters [Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0707699,1.0707699,0.0,0.0), Dual{ForwardDif

     2     1.814787e+00     5.672059e-03
 * Current step size: 0.15930836
 * time: 172.1668939590454
 * g(x): Float32[-0.00019501134, 1.0135125f-5, 0.005672059]
 * x: Float32[0.06837795, -4.0081133f-5, -0.33532402]


[ Info: 20220508 17:10:12 training model with parameters [Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.057375,1.057375,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.99974656,0.0,0.99974656,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.66611916,0.0,0.0,0.66611916)]
Progress: 100%|███████████████████████████| Time: 0:00:04 ( 0.44 μs/it)
[ Info: 20220508 17:10:34 training model with parameters [Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0630815,1.0630815,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.99983776,0.0,0.99983776,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.6866397,0.0,0.0,0.6866397)]
Progress: 100%|███████████████████████████| Time: 0:00:04 ( 0.45 μs/it)


...

    20     1.814489e+00     1.364076e-06
 * Current step size: 2.6075046
 * time: 1459.9649980068207
 * g(x): Float32[3.9563918f-7, 4.4214424f-11, -1.3640758f-6]
 * x: Float32[-0.15989731, -10.340025, -0.4399279]


[ Info: 20220508 17:31:40 training model with parameters [Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.8522434,0.8522434,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(2.437815e-5,0.0,2.437815e-5,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.6440905,0.0,0.0,0.6440905)]
Progress: 100%|███████████████████████████| Time: 0:00:04 ( 0.44 μs/it)
[ Info: 20220508 17:32:02 training model with parameters [Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.8522918,0.8522918,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(7.897093e-6,0.0,7.897093e-6,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.64412093,0.0,0.0,0.64412093)]
Progress: 100%|███████████████████████████| Time: 0:00:04 ( 0.44 μs/it)
[ Info: 20220508 17:32:23 training model with parameters [Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.8522564,0.8522564,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.7995784e-5,0.0,1.799

    21     1.814489e+00     7.983481e-07
 * Current step size: 2.07719
 * time: 1524.253702878952
 * g(x): Float32[-1.5771829f-7, 2.2963187f-11, 7.983481f-7]
 * x: Float32[-0.15986782, -10.925373, -0.43990332]


[ Info: 20220508 17:32:45 training model with parameters [Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.8522527,0.8522527,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.4079447e-5,0.0,1.4079447e-5,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.64409596,0.0,0.0,0.64409596)]
Progress: 100%|███████████████████████████| Time: 0:00:04 ( 0.45 μs/it)
[ Info: 20220508 17:33:06 training model with parameters [Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.85223794,0.85223794,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(5.275278e-6,0.0,5.275278e-6,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.64408505,0.0,0.0,0.64408505)]
Progress: 100%|███████████████████████████| Time: 0:00:04 ( 0.44 μs/it)
[ Info: 20220508 17:33:27 training model with parameters [Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.85224056,0.85224056,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(6.277633e-6,0.

    22     1.814489e+00     8.385715e-07
 * Current step size: 4.291169
 * time: 1588.694303035736
 * g(x): Float32[2.8909088f-7, 7.647784f-12, -8.385715f-7]
 * x: Float32[-0.15988646, -11.978518, -0.43992147]


In [9]:
@info "The optimal [λ_u, λ_a, λ_w] is $λ, found in " *
      repr(Optim.f_calls(res)) *
      " function calls"

[ Info: 20220508 17:33:49 The optimal [λ_u, λ_a, λ_w] is Float32[0.85224056, 6.277633f-6, 0.644087], found in 75 function calls


In [10]:
empty!(memoize_cache(get_weights))
stop_criteria = convergence_stopper(1e-6, max_iters = 16)
u, a = train_model(training, λ..., stop_criteria);

[ Info: 20220508 17:33:49 training model with parameters [0.85224056, 6.277633e-6, 0.644087]
Progress: 100%|███████████████████████████| Time: 0:00:04 ( 0.46 μs/it)


## Inference

In [11]:
model(users, items) = make_prediction(users, items, u, a);

In [12]:
write_predictions(model; residual_alphas = residual_alphas);

[ Info: 20220508 17:34:14 training set: RMSE 1.2844305 MAE 0.96078026 R2 0.46220618
Progress: 100%|███████████████████████████| Time: 0:00:04 ( 0.45 μs/it)
[ Info: 20220508 17:34:27 training set weighted-loss: RMSE 1.2921827 MAE 0.96010107 R2 0.48657387
[ Info: 20220508 17:34:28 validation set: RMSE 1.3195703 MAE 0.9857642 R2 0.40952003
[ Info: 20220508 17:34:29 validation set weighted-loss: RMSE 1.3469635 MAE 1.0024703 R2 0.39467692


In [13]:
write_params(Dict("u" => u, "a" => a, "λ" => λ));